# __Brough Lecture Notes: GARCH Models - Estimation via MLE__

<br>

Finance 5330: Financial Econometrics <br>
Tyler J. Brough <br>
Last Updated: March 28, 2019 <br>
<br>
<br>

These notes are based in part on the excellent monograph [Introduction to Python for Econometrics, Statistics, and Data Analysis](https://www.kevinsheppard.com/images/b/b3/Python_introduction-2016.pdf) by the econometrician Kevin Sheppard of Oxford Univeristy. Many thanks to Dr. Sheppard for making his lecture material publically available. 

<br>

## Estimating GARCH Models

The standard procedure to fit GARCH models to historical returns time series data is to implement a numerical _maximum likelihood estimation_ (MLE) method. 

<br>

__NB:__ though [Bayesian](https://www.springer.com/us/book/9783540786566) methods have been shown to be superior!

<br>

The typical setup is as follows: 

* Continuously compounded returns are assumed to have a conditionally normal distribution $N(0, \sigma_{t})$

* We can estimate the GARCH parameter weights via a numerical optimization routine such as Nelder-Mead or Newton-Raphson.

* That is, the numerical routine searches for the parameter values that maximizes the value of the likelihood function.

<br>

Under the normality assumption the probility density of $\epsilon_{t}$, conditional on $\sigma_{t}$, is 

<br>

$$
f(\epsilon | \sigma_{t}) = \frac{1}{\sqrt{2\pi \sigma_{t}}} e^{-0.5  \frac{\epsilon_{t}^{2}}{\sigma_{t}}}
$$

<br>

Since the $\epsilon_{t}$ are conditionally independent, the probability of observing the actual returns that are observed is  the product of the probabilities, this is given by the likelihood function:

<br>

$$
\prod\limits_{t=1}^{T} f(\epsilon_{t} | \sigma_{t}) = \prod\limits_{t=1}^{T} \left( \frac{1}{\sqrt{2\pi \sigma_{t}}} e^{-0.5  \frac{\epsilon_{t}^{2}}{\sigma_{t}}} \right)
$$

<br>

For the GARCH(1,1) model, $\sigma_{t}$ is a function of $\omega$, $\alpha$, and $\beta$. The MLE will select values for these parameters $\hat{\omega}$, $\hat{\alpha}$, and $\hat{\beta}$ - that maximize the value of the probability of observing the returns we actually historically did observe. 

<br>

Typically, it is easiest to maximize the value of the log-likelihood function as follows: 

<br>

$$
\sum\limits_{t=1}^{T} \left[ -0.5 \ln{(\sigma_{t})} - 0.5 \frac{\epsilon_{t}^{2}}{\sigma_{t}}\right]
$$

We can omit the term $-0.5 \ln{(2 \pi)}$ since it does not affect the solution. Though sometimes it is left in. 

<br>

We can implement this MLE estimation in Python by utilizing the [optimize](https://docs.scipy.org/doc/scipy/reference/optimize.html) module in [Scipy](https://docs.scipy.org/doc/scipy/reference/index.html), which contains a host of numerical optimization routines. 

<br>

First, we need to define a function to implement the log-likelihood function.

<br>

In [344]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

import seaborn
from numpy import size, log, exp, pi, sum, diff, array, zeros, diag, mat, asarray, sqrt, copy
from numpy.linalg import inv

import scipy.optimize as opt

In [345]:
def garch_likelihood(params, data, sigma2, out=None):
    mu = params[0]
    omega = params[1]
    alpha = params[2]
    beta = params[3]
    
    T = size(data, 0)
    eps = data - mu
    
    for t in range(1, T):
        sigma2[t] = omega + alpha * eps[t-1]**2 + beta * sigma2[t-1]
        
    lls = 0.5 * (log(2 * pi) + log(sigma2) + eps**2/sigma2)
    ll = sum(lls)
    
    if out is None:
        results = ll
    else:
        results = (ll, lls, copy(sigma2))
        
    return results

<br>

We should begin with data simulated from the model as a first-run test case. So, let's import the simulation code.

<br>

In [346]:
## Set GARCH parameters (these might come from an estimated model)
mu = log(1.15) / 252
w = 10.0**-6
a = 0.085
b = 0.905

In [347]:
np.sqrt(w / (1.0 - a - b)) * np.sqrt(252)

0.15874507866387536

In [348]:
def simulate_garch(parameters, numObs):
    
    ## extract the parameter values
    mu = parameters[0]
    w = parameters[1]
    a = parameters[2]
    b = parameters[3]
    
    ## initialize arrays for storage
    z = np.random.normal(size=(numObs + 1))
    q = zeros((numObs + 1))
    r = zeros((numObs + 1))
    
    ## fix initial values 
    q[0] = w / (1.0 - a - b)
    r[0] = mu + z[0] * sqrt(q[0])
    e = (r[0] - mu) 
    
    ## run the main simulation loop
    for t in range(1, numObs + 1):
        q[t] = w + a * (e * e) + b * q[t-1]
        r[t] = mu + z[t] * sqrt(q[t])
        e = (r[t] - mu) 
        
    ## return a tuple with both returns and conditional variances
    return (r, q)

In [349]:
## number of trading days per year
numObs = 2500

## daily continuously compounded rate of return correpsonding to 15% annual
#mu = log(1.15) / 252
#mu = 0.0

## drift and GARCH(1,1) parameters in an array
params = array([mu, w, a, b])

## run the simulation
r, s = simulate_garch(params, numObs)

<br>

Okay, now we will set up the estimation of the model on this simulated data. The goal is to see if the numerical search routine embedded in the maximum likelihood estimation
can recapture the preset parameter weights given above. 

<br>

It is important to test our software with a known scenario like this before we take the model to real-world data first as a sanity check. Otherwise we could end up chasing our tails for hours and hours without effect. 

<br>

The numerical algorithm requires good starting values as initial conditions for the search to begin. Here we will follow Sheppard. (Notice that this feels "prior" like, as in the Bayesian sense) A more data-based way to do this is to use some kind of grid search to find values that have "small" log-likelihood values. 

<br>

In [350]:
r.mean()

0.0005889303527271982

In [351]:
#begVals = array([r.mean(), r.var() * .01, .09, .90])
begVals = array([mu, w, a, b])

In [364]:
finfo = np.finfo(np.float64)
bounds = [(-10 * r.mean(), 10 * r.mean()), (finfo.eps, 2 * r.var()), (0.0, 1.0), (0.0, 1.0)]
#results = opt.minimize(fun=garch_likelihood, x0=begVals, args=(r,s), method='L-BFGS-B', bounds=bounds)
#results = opt.minimize(fun=garch_likelihood, x0=begVals, args=(r,s), method='Nelder-Mead')
results = opt.minimize(fun=garch_likelihood, x0=begVals, args=(r,s), method='BFGS')

In [365]:
results

      fun: -8164.241691969553
 hess_inv: array([[ 2.61645762e-08,  9.40425497e-13, -1.43341977e-08,
        -5.20680878e-09],
       [ 9.40425497e-13,  1.52534302e-13, -7.53473757e-09,
         3.76315979e-09],
       [-1.43341977e-08, -7.53473757e-09,  4.80530085e-04,
        -2.46500899e-04],
       [-5.20680878e-09,  3.76315979e-09, -2.46500899e-04,
         1.27968430e-04]])
      jac: array([6.10351562e-05, 3.99658203e-01, 6.10351562e-05, 6.10351562e-05])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 353
      nit: 11
     njev: 57
   status: 2
  success: False
        x: array([6.17962579e-04, 1.09401589e-06, 8.84289107e-02, 9.02360284e-01])

In [366]:
vals = results['x']
print(f"The estimated mean return is: {vals[0] : 0.8f} vs {mu : 0.8f}")
print(f"The estimated omega value is: {vals[1] : 0.8f} vs {w : 0.8f}")
print(f"The estimated alpha value is: {vals[2] : 0.8f} vs {a : 0.08f}")
print(f"The estimated beta values is: {vals[3] : 0.8f} vs {b : 0.08f}")

The estimated mean return is:  0.00061796 vs  0.00055461
The estimated omega value is:  0.00000109 vs  0.00000100
The estimated alpha value is:  0.08842891 vs  0.08500000
The estimated beta values is:  0.90236028 vs  0.90500000


In [363]:
r.std(ddof=1) * np.sqrt(252)

0.16500859496922918

<br>

___So much depends upon those initial guesses!___

<br>

## GARCH Estimation on IBM Returns from January 1999 to 2003

In [163]:
def garch_constraints(params, data, sigma2, out=None):
    omega = params[1]
    alpha = params[2]
    beta = params[3]
    
    return array([[omega], [alpha], [beta], [1.0 - alpha - beta]])

In [164]:
ibm = pd.read_csv("./data/IBM-1999-2003.csv", parse_dates=True, index_col=1)

In [165]:
ibm.head()

,PERMNO,TICKER,COMNAM,PERMCO,PRC,RET,CFACPR,RETX,sprtrn
date,,,,,,,,,
1999-01-04,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,183.0000,-0.007458,2,-0.007458,-0.000919
1999-01-05,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,189.6250,0.036202,2,0.036202,0.013582
1999-01-06,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,188.7500,-0.004614,2,-0.004614,0.022140
1999-01-07,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,190.1875,0.007616,2,0.007616,-0.002051
1999-01-08,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,187.5625,-0.013802,2,-0.013802,0.004221


In [166]:
ibm.tail()

,PERMNO,TICKER,COMNAM,PERMCO,PRC,RET,CFACPR,RETX,sprtrn
date,,,,,,,,,
2003-12-24,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,92.27,-0.005604,1,-0.005604,-0.001807
2003-12-26,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,92.90,0.006828,1,0.006828,0.001691
2003-12-29,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,93.52,0.006674,1,0.006674,0.012401
2003-12-30,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,92.63,-0.009517,1,-0.009517,0.000144
2003-12-31,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,92.68,0.000540,1,0.000540,0.002055


In [167]:
prc = np.abs(ibm.PRC.values)

In [168]:
prc.min()

55.07

In [169]:
prc.max()

246.0

In [182]:
ret = ibm.sprtrn.values
#ret = ibm.RETX.values

In [183]:
#begVals = array([mu, w, a, b])
r = 100 * ret
sigma = np.ones_like(r) * r.var()
args = (r, sigma)

begVals = array([0.0, 10**-6, .01, .9])
finfo = np.finfo(np.float64)
bounds = [(-10 , 10), (finfo.eps, 2 * r.var()), (0.0, 1.0), (0.0, 1.0)]

#estimates = fmin_slsqp(garch_likelihood, begVals, f_ieqcons=garch_constraints, bounds=bounds, args=args)
#bob = minimize(fun=garch_likelihood, x0=begVals, args=(r,s), method='SLSQP', bounds=bounds, constraints=garch_constraints)
bob = minimize(fun=garch_likelihood, x0=begVals, args=args, method='L-BFGS-B', bounds=bounds) #, constraints=garch_constraints)

In [184]:
bob

      fun: 2081.4537306219727
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.0005457 , 0.00391083, 0.00336513, 0.00545697])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 320
      nit: 33
   status: 0
  success: True
        x: array([0.02738401, 0.04148747, 0.07527011, 0.90160256])

In [185]:
res = bob['x']

print(f"The estimated mean return is: {res[0] : 0.8f}")
print(f"The estimated omega value is: {res[1] : 0.8f}")
print(f"The estimated alpha value is: {res[2] : 0.8f}")
print(f"The estimated beta values is: {res[3] : 0.8f}")

The estimated mean return is:  0.02738401
The estimated omega value is:  0.04148747
The estimated alpha value is:  0.07527011
The estimated beta values is:  0.90160256


In [187]:
1.0 - res[2] - res[3]

0.023127329801457486

In [192]:
np.sqrt(res[1] / (1.0 - res[2] - res[3])) * np.sqrt(252)

21.261601831419906

In [196]:
np.std(r, ddof=1) * np.sqrt(252)

21.22957178731841

## GJR-GARCH(1,1,1) Model Estimation via MLE